# Introduction to Pandas
### By **[NimbleBox](https://www.nimblebox.ai)**

[<img src="./assets/nbx.jpeg" alt="NimbleBox.ai logo" width="600"/>](https://www.nimblebox.ai)

This notebook contains parts from [Pandas](https://pandas.pydata.org/).


## Introduction

According to the official [readme](https://github.com/pandas-dev/pandas), **pandas** is a python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive.

In this tutorial, we will cover:

* **Basics of Pandas**: Introduction Pandas Objects, creation of commonly used Pandas Objects.
* **Operations on Data**: 
* **Aggregations**: Various function used to aggregate for NumPy arrays

## Getting Started

Typically we start by importing the python package as follows.

In [ ]:
import pandas as pd

## Creating data

The primary two components of pandas are the `Series` and `DataFrame`. A `Series` is essentially a column, and a `DataFrame` is a multi-dimensional table made up of a collection of `Series`.

<img width="640" height="480" src="https://storage.googleapis.com/lds-media/images/series-and-dataframe.width-1200.png" />

DataFrames and Series are quite similar in that many operations that you can do with one you can do with the other, such as filling in null values and calculating the mean.

### DataFrame

A DataFrame is a table. It contains an array of individual entries, each of which has a certain value. Each entry corresponds to a row (or record) and a column.

For example, consider the following simple DataFrame from the `data`:

In [ ]:
data = {
    'Python': ['Guido van Rossum', '.py', 1991],
    'Javascript': ['Brendan Eich', '.js', 1995]
}

In [ ]:
programming = pd.DataFrame(data)
programming

The `Index` of this DataFrame was given to us on creation as the numbers 0-2, but we could also create our own when we initialize the DataFrame.

In [ ]:
programming = pd.DataFrame(data, index=['Creator', 'Extention', 'Year'])
programming

Now we can use `loc` to locate language's attributes based on a given Index.

In [ ]:
programming.loc['Creator']

There's more on locating and extracting data from the DataFrame which are described in the [pandas documentation]()

## Reading Data

It’s quite simple to load data from various file formats into a DataFrame. Let's see some examples of how to read data from CSV, JSON, or SQL files

### Reading data from CSVs

In [ ]:
df = pd.read_csv('./assets/programming.csv')
df

CSVs don't have indexes like our DataFrames, so all we need to do is just designate the `index_col` when reading:

In [ ]:
df = pd.read_csv('./assets/programming.csv', index_col=0)
df

Here we're setting the index to be column zero.

### Reading data from JSON

In [ ]:
df = pd.read_json('./assets/programming.json')
df

Notice this time our index came with us correctly since using JSON allowed indexes to work through nesting. Pandas will try to figure out how to create a DataFrame by analyzing structure of your JSON, and sometimes it doesn't get it right. Often you'll need to set the orient keyword argument depending on the structure, so check out [read_json docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html) about that argument to see which orientation you're using.

### Reading data from a SQL database

If you’re working with data from a SQL database you need to first establish a connection using an appropriate Python library, then pass a query to pandas. Here we'll use SQLite to demonstrate.

First, we need `pysqlite3` installed, so run this command in your terminal:

```
$ pip install pysqlite3
```


`sqlite3` is used to create a connection to a database which we can then use to generate a DataFrame through a `SELECT` query.

So first we'll make a connection to a SQLite database file:

In [ ]:
import sqlite3
con = sqlite3.connect("./assets/database.db")
df = pd.read_sql_query("SELECT * FROM programming", con)

df

Just like with CSVs, we could pass `index_col='index'`, but we can also set an index after-the-fact:

In [ ]:
df = df.set_index('index')
df

In fact, we could use `set_index()` on any DataFrame using any column at any time. Indexing Series and DataFrames is a very common task, and the different ways of doing it is worth remembering.

### Converting back to a CSV, JSON, or SQL

So after extensive work on cleaning your data, you’re now ready to save it as a file of your choice. Similar to the ways we read in data, pandas provides intuitive commands to save it:

In [ ]:
df.to_csv('./assets/programming.csv')

df.to_json('./assets/programming.json')

df.to_sql('./assets/programming_1', con)

When we save JSON and CSV files, all we have to input into those functions is our desired filename with the appropriate file extension. With SQL, we’re not creating a new file but instead inserting a new table into the database using our `con` variable from before.

### Most used DataFrame operations of Pandas

DataFrames possess hundreds of methods and other operations that are crucial to any kind of data analysis. Let us look at some of the most used operations that perform simple transformations of your data and those that provide fundamental statistical analysis.

Let us first load the movie dataset from a CSV and designating the movie titles to be our index:

In [ ]:
movies_df = pd.read_csv("./assets/IMDB-Movie-Data.csv")

In [ ]:
movies_df

#### Previewing our Datasets

The first thing to do when opening a new dataset is print out a few rows to keep as a visual reference. We accomplish this with `.head()` or by using `.tail()`:

In [ ]:
movies_df.head()

In [ ]:
movies_df.tail()

By default both `.head()` and `.tail()` outputs the first and the last 5 rows respectively of our DataFrame. Typically when we load in a dataset, we like to view the first five rows to see what's under the hood like the names of each column, the index, and examples of values in each row

### Getting Info about the Data

The `.info()` operation provides the essential details about your dataset, such as the number of rows and columns, the number of non-null values, what type of data is in each column, and how much memory your DataFrame is using.

In [ ]:
movies_df.info()

Another useful attribute is `.shape`, which outputs just a tuple of (rows, columns).

In [ ]:
movies_df.shape

### Handling Duplicates in our Dataset

It is always a good practice to verify that our dataset don't have any duplicate rows. Now we can try dropping duplicates:

In [ ]:
movies_df = movies_df.drop_duplicates()

In [ ]:
movies_df.shape

The `drop_duplicates()` method will return a copy of our DataFrame, and but the duplicates gets removed. With the `.shape` operation we can see that duplicate rows are removed.

### Column cleanup

Many times datasets will have verbose column names with symbols, upper and lowercase words, spaces, and typos. To make selecting data by column name easier we can spend a little time cleaning up their names.

In [ ]:
movies_df.columns

We can use the `.rename()` method to rename certain or all columns via a `dict`. Let's say we want us to rename some of the names of these columns:

In [ ]:
movies_df.rename(columns={
        'Runtime (Minutes)': 'Runtime', 
        'Revenue (Millions)': 'Revenue_millions'
    }, inplace=True)


movies_df.columns

### Working with Missing Values

When exploring data, you’ll most likely encounter missing or null values, which are essentially placeholders for non-existent values. Most commonly you'll see Python's None or NumPy's np.nan, each of which are handled differently in some situations.

There are two options in dealing with nulls:

1. Get rid of rows or columns with nulls
2. Replace nulls with non-null values, a technique known as imputation

Let's calculate to total number of nulls in each column of our dataset. The first step is to check which cells in our DataFrame are null:

In [ ]:
movies_df.isnull().head(10)

The function `.isnull()` just by iteself isn't very useful, and is usually used in conjunction with other methods, like `sum()`.

We can see now that our data has **128 missing** values for `revenue_millions` and **64** missing values for `metascore`.

#### Removing null values

Data Scientists and Analysts regularly face the dilemma of dropping or imputing null values, and is a decision that requires intimate knowledge of your data and its context. Overall, removing null data is only suggested if you have a small amount of missing data.

In [ ]:
movies_df.dropna()

So in the case of our dataset, this operation would remove **128** rows where `revenue_millions` is null and **64** rows where `metascore` is null.

#### Imputation

Imputation is a conventional feature engineering technique used to keep valuable data that have null values.

There may be instances where dropping every row with a null value removes too big a chunk from your dataset, so instead we can impute that null with another value, usually the mean or the median of that column.

Let's look at imputing the missing values in the revenue_millions column. First we'll extract that column into its own variable:


In [ ]:
revenue = movies_df['Revenue_millions']

Now when we select columns of a DataFrame, we use brackets just like if we were accessing a Python dictionary. `revenue` now contains a Series:

In [ ]:
revenue.head()

We'll impute the missing values of revenue using the mean. Here's the mean value:

In [ ]:
revenue_mean = revenue.mean()
revenue_mean

With the mean, let's fill the nulls using `fillna()`:

In [ ]:
revenue.fillna(revenue_mean, inplace=True)

We have now replaced all nulls in `revenue` with the mean of the column. Notice that by using `inplace=True` we have actually affected the original `movies_df`:

In [ ]:
movies_df.isnull().sum()

Imputing an entire column with the same value like this is a basic example.

#### Understanding your variables

Using `describe()` on an entire DataFrame we can get a summary of the distribution of continuous variables:

In [ ]:
movies_df.describe()

Understanding which numbers are continuous also comes in handy when thinking about the type of plot to use to represent your data visually.

`.describe()` can also be used on a categorical variable to get the count of rows, unique count of categories, top category, and freq of top category:

In [ ]:
movies_df['Genre'].describe()

This tells us that the genre column has **207** unique values, the top value is Action/Adventure/Sci-Fi, which shows up **50** times (freq).

`.value_counts()` can tell us the frequency of all values in a column:

#### Relationships between continuous variables
By using the correlation method `.corr()` we can generate the relationship between each continuous variable:

In [ ]:
movies_df.corr()

Correlation tables are a numerical representation of the bivariate relationships in the dataset.

### Selecting rows and columns
Using .loc[] and.iloc[] you can select particular rows and columns in a dataframe

In [ ]:
movies_df.iloc[45]

In [ ]:
movies_df_2=movies_df.set_index('Title')

In [ ]:
movies_df_2.head()

In [ ]:
movies_df_2.loc['Sing']

In [ ]:
movies_df.iloc[[5,11,15,29]]

### Operations on dataframe

In [ ]:
movies_df[movies_df['Runtime']>170]

In [ ]:
movies_df[(movies_df['Runtime']>150) & (movies_df['Rating']>8.5)]

#### Group DataFrame using a mapper or by a Series of columns using .groupby()

In [ ]:
directors_df=movies_df.groupby('Director').mean().reset_index()
directors_df.head()

In [ ]:
directors_df[directors_df['Rating']>8.3]

In [ ]:
movies_df.groupby('Year')['Rating'].mean()

In [ ]:
movies_df.groupby('Year')['Rating'].max()

#### Apply a function along an axis of the DataFrame using .apply()

In [ ]:
def times10(x): 
    return 10*x
movies_df['Rating']=movies_df['Rating'].apply(times10)

In [ ]:
movies_df.head()